> # Spam Classification

this is a spam classification, with out using any machine learning libarie.

## I- Packages

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

%matplotlib widget

## II- Data

In [3]:
data = pd.read_csv('./data/emails.csv')

data.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [4]:
x_data = data.drop('Prediction', axis=1).drop('Email No.', axis=1).values
y_data = data['Prediction'].values

### III- data visualization

In [11]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Columns: 3002 entries, Email No. to Prediction
dtypes: int64(3001), object(1)
memory usage: 118.5+ MB


,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
count,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,...,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000,5172.000000
mean,6.640565,6.188128,5.143852,3.075599,3.124710,2.627030,55.517401,2.466551,2.024362,10.600155,...,0.005027,0.012568,0.010634,0.098028,0.004254,0.006574,0.004060,0.914733,0.006961,0.290023
std,11.745009,9.534576,14.101142,6.045970,4.680522,6.229845,87.574172,4.314444,6.967878,19.281892,...,0.105788,0.199682,0.116693,0.569532,0.096252,0.138908,0.072145,2.780203,0.098086,0.453817
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,12.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,3.000000,1.000000,1.000000,2.000000,1.000000,28.000000,1.000000,0.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,7.000000,4.000000,3.000000,4.000000,2.000000,62.250000,3.000000,1.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,210.000000,132.000000,344.000000,89.000000,47.000000,77.000000,1898.000000,70.000000,167.000000,223.000000,...,4.000000,7.000000,2.000000,12.000000,3.000000,4.000000,3.000000,114.000000,4.000000,1.000000


## IV- Logistic Regression

### 1- Buiding the model

- ### sigmoid function $ \sigma$
$$ \sigma(x) = \frac{1}{1+e^{-x}}$$

In [12]:
def sigmoid(x):
    s = 1 /(1 + np.exp(-x))        
    return s

- ### $loss$ function 
we will use the log loss function

$\therefore  loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$

$\therefore f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \sigma(w \cdot \mathbf{x}^{(i)}+b)$

$\therefore
(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x} + b = w_0x^{(i)}_0 + ... + w_{n-1}x^{(i)}_{n-1} + b
$

$
\rightarrow
\begin{cases} 
loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = -\log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) & \text{if } y^{(i)} = 1 \\
loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = -\log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) & \text{if } y^{(i)} = 0
\end{cases}

$\rightarrow
loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) $



In [14]:
def lost(x,y,w,b):   
    r = ((-y) * np.log(sigmoid(np.dot(x,w)+b))) - ((1 - y) * np.log(1 - sigmoid(np.dot(x,w)+b)))
    return r

- ### $Cost$ function 
$$ Cost(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] $$


In [13]:
def cost(x, y, w, b, *argv):
 
    m = x.shape[0]
    err = 0.0
    
    for i in range(m):
        
        err += lost(x[i],y[i],w,b)
    
    total_cost = err/m   

    return total_cost


- ### Partial Derivatives 
$$
\frac{\partial J(\mathbf{w},b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{1}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{2}
$$

In [15]:
def partial_derivatives(X, y, w, b, *argv): 
    """
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
    """
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar
        
    return dj_db, dj_dw 

- ### Gradient Descent
$$\begin{align*}\newline \; & b := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \newline       \; & w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \; & \newline \end{align*}$$

In [17]:
def gradient_descent(X, y, w_in,b_in, alpha, num_iters, lambda_=0): 
    
    m = len(X)
    
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        dj_db, dj_dw = partial_derivatives(X, y, w_in, b_in, lambda_)   

        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        if i<100000:  
            cost =  cost(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history 

- ### Prediction

In [18]:
def predict(x, w, b): 
   
    m, n = x.shape   
    p = np.zeros(m)
  
    for i in range(m):   
        z_wb = 0
        for j in range(n): 
            z_wb_ij = x[i, j] * w[j]
            z_wb += z_wb_ij
        z_wb += b
        f_wb = sigmoid(z_wb)
        p[i] = f_wb >= 0.5
        
    return p

### 2- Training the model

In [23]:
w_in = np.zeros(x_data.shape[1])

gradient_descent(x_data, y_data, w_in,b_in, 0.01, 1000,0)

TypeError: gradient_descent() takes from 5 to 6 positional arguments but 7 were given